In [1]:
import awswrangler as awr
import pandas as pd

df

In [2]:
file_path = r"C:\Users\raphael.almeida\Documents\Processos\historico_placas\sql\all_boards.sql"
with open (file_path, 'r') as r:
    query = r.read()

df = awr.athena.read_sql_query(query, database='silver')





In [3]:
# Identificar as placas que possuem pelo menos os status "CANCELADO" e "ATIVO"
placas_com_ambos_status = df.groupby('placa')['status_conjunto'] \
                            .apply(lambda x: {'CANCELADO', 'ATIVO'}.issubset(set(x))) \
                            .loc[lambda x: x].index

# Filtrar o DataFrame completo para manter apenas essas placas
df_filtrado = df[df['placa'].isin(placas_com_ambos_status)]

df_filtrado

,coverage,beneficio,placa,chassi,id_placa,matricula,conjunto,data_registro,data_cancelamento_conjunto,data_ativacao_conjunto,status_conjunto,data_ativacao_beneficio,status_beneficio,empresa,data_atualizacao
2,65813,ASSISTÊNCIA 24 HORAS,EZL8C82,9BM693186CB833677,27399,12505,10636,2024-08-23,None,2024-10-04,ATIVO,2024-10-04,ATIVO,Stcoop,2025-02-06
3,65811,REPARAÇÃO A TERCEIROS,EZL8C82,9BM693186CB833677,27399,12505,10636,2024-08-23,None,2024-10-04,ATIVO,2024-10-04,ATIVO,Stcoop,2025-02-06
18,34950,REPARAÇÃO A TERCEIROS,ATT2544,9BM958461AB707138,17412,12529,5604,2024-02-02,2025-02-03,2024-02-05,CANCELADO,2024-02-05,CANCELADO,Stcoop,2025-02-04
39,35398,ASSISTÊNCIA 24 HORAS,IVZ7B35,9BM958443DB941927,19050,12561,5679,2024-02-05,2024-02-14,2024-02-05,CANCELADO,2024-02-05,CANCELADO,Stcoop,2024-02-14
40,35397,REPARAÇÃO A TERCEIROS,IVZ7B35,9BM958443DB941927,19050,12561,5679,2024-02-05,2024-02-14,2024-02-05,CANCELADO,2024-02-05,CANCELADO,Stcoop,2024-02-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685835,578395,REPARAÇÃO OU REPOSIÇÃO DO VEÍCULO,SEM-PLACA,953698TK8PR040842,155157,47575,124765,2023-03-13,None,None,DESISTÊNCIA,2023-03-13,NOVO,Segtruck,2023-03-14
685865,659802,REPARAÇÃO OU REPOSIÇÃO DO VEÍCULO,BCE8G25,9BSR6X400J3930051,156410,47583,137670,2024-03-22,2025-04-07,2024-03-22,FINALIZADO,2024-03-22,ATIVO,Segtruck,2024-05-03
685866,583442,REPARAÇÃO OU REPOSIÇÃO DO VEÍCULO,BCE8G25,9BSR6X400J3930051,156410,47583,125562,2023-03-27,2024-03-22,2023-03-28,CANCELADO,2023-03-28,CANCELADO,Segtruck,2024-03-22
685894,576886,VIDROS,AXR1A18,9BM958443DB922947,70180,47601,124550,2023-03-09,2023-08-14,2023-03-26,CANCELADO,2023-03-26,CANCELADO,Segtruck,2023-08-14


In [4]:
# Ordenar por placa e data_registro para garantir a ordem cronológica
df_sorted = df.sort_values(['placa', 'data_registro'])

# Função para verificar se "ATIVO" aparece antes de "CANCELADO"
def ativo_antes_cancelado(grupo):
    status = grupo['status_conjunto'].values
    return 'ATIVO' in status and 'CANCELADO' in status and list(status).index('ATIVO') < list(status).index('CANCELADO')

# Identificar placas que atendem à condição
placas_ativo_antes_cancelado = df_sorted.groupby('placa').filter(ativo_antes_cancelado)['placa'].unique()

print("Placas com status ATIVO antes de CANCELADO:")
print(placas_ativo_antes_cancelado)

Placas com status ATIVO antes de CANCELADO:
<StringArray>
['ABI6E48', 'ABI6F58', 'ABJ2425', 'ABZ6B69', 'AFD7H68', 'AKO2D69', 'ATM8H72',
 'AUA8I08', 'AUB7E37', 'AXT1B71', 'AYC6B19', 'AYZ5A44', 'BCN7J85', 'BCN7J87',
 'BDP1G34', 'CSK3I11', 'FDC9I09', 'FTP3G46', 'GVE3F38', 'HLY4B80', 'ICV2143',
 'IVY4D91', 'JZM3A67', 'MGY2G95', 'MXE5D35', 'NKO1H24', 'NRM7970', 'NRM7971',
 'ONO1D65', 'ONO1G55', 'ONO2H35', 'OOM5C63', 'OOM5C64', 'OOM5E81', 'OOM5E82',
 'QAJ3D40', 'QAJ3E40', 'QAU2A09', 'QAU2A19', 'QCS6268', 'QHL3B99', 'QJZ7772',
 'QTZ9F45', 'RDU7E35', 'RHK6H06', 'RLI4F53', 'RRL1C06', 'RRU3F97', 'RRU3G87',
 'RRU3I27', 'RXK7B77', 'RYJ2E31', 'SEG7H19', 'SFH2G93', 'SLI3J78', 'SLI4C28',
 'SLI4D88', 'SXN6I39', 'TAM3E07']
Length: 59, dtype: string


In [5]:
df1 = df[df['placa']=='SLI4D88']
df1.sort_values('data_registro', ascending=False)

,coverage,beneficio,placa,chassi,id_placa,matricula,conjunto,data_registro,data_cancelamento_conjunto,data_ativacao_conjunto,status_conjunto,data_ativacao_beneficio,status_beneficio,empresa,data_atualizacao
107870,55618,CASCO (R/SR),SLI4D88,9A9SRCAB2R1DK4028,4850,5069,7519,2024-12-26,2025-05-05,2024-12-30,CANCELADO,2024-12-30,CANCELADO,Viavante,2025-05-05
138688,33419,CASCO (R/SR),SLI4D88,9A9SRCAB2R1DK4028,4850,3426,4728,2024-10-30,None,2024-10-29,ATIVO,2024-10-31,CANCELADO,Viavante,2025-05-06
147189,33420,RASTREADOR REBOQUE/SEMIRREBOQUE,SLI4D88,9A9SRCAB2R1DK4028,4850,3426,4728,2024-10-30,None,2024-10-29,ATIVO,2024-10-31,CANCELADO,Viavante,2025-05-06


DF_SETS

df_set

In [6]:
file_path = r"C:\Users\raphael.almeida\Documents\Processos\historico_placas\sql\all_boards_sets.sql"
with open (file_path, 'r') as r:
    query = r.read()

df_sets = awr.athena.read_sql_query(query, database='silver')


In [7]:
df_sets.sort_values('placa')

,placa,chassi,id_placa,matricula,conjunto,data_registro,data_cancelamento_conjunto,data_ativacao_conjunto,status_conjunto,empresa,data_atualizacao
211385,AAT8826,67639,32750,46009,114637,2022-09-09,None,None,NOVO,Segtruck,1754-08-30 22:43:41.128654848
222095,AAT8826,67639,32750,36372,57777,2020-06-10,None,2020-06-10,SUBSTITUÍDO,Segtruck,2020-08-26 00:00:00.000000000
201572,AAT8826,67639,32750,36372,59977,2020-07-22,2021-05-20,2020-07-22,CANCELADO,Segtruck,2021-05-20 00:00:00.000000000
160752,AKC1D93,9A90710202SAC6332,18187,12145,56763,2020-05-22,2020-12-21,2020-05-23,CANCELADO,Segtruck,2020-12-21 00:00:00.000000000
142615,AKC1D93,9A90710202SAC6332,18187,12145,22584,2019-05-24,None,2019-05-25,RENOVAÇÃO,Segtruck,2020-05-24 00:00:00.000000000
...,...,...,...,...,...,...,...,...,...,...,...
56680,ZZZ9999,528LKACKDXPPD244,186298,2149,2833,2024-09-03,None,2024-09-03,SUBSTITUÍDO,Viavante,2025-04-22 00:00:00.000000000
88650,¨@%NBY5133,%¨#%$#¨%$82G5C054546,8255,1798,12952,2018-05-28,None,2018-05-29,RENOVAÇÃO,Segtruck,2019-05-23 00:00:00.000000000
176468,¨@%NBY5133,%¨#%$#¨%$82G5C054546,8255,41355,76806,2021-05-25,2022-06-12,2021-05-27,FINALIZADO,Segtruck,2023-05-23 00:00:00.000000000
75221,¨@%NBY5133,%¨#%$#¨%$82G5C054546,8255,1798,22573,2019-05-23,None,2019-05-25,RENOVAÇÃO,Segtruck,2020-05-24 00:00:00.000000000


In [11]:
df_sets['status_conjunto'].unique()

<StringArray>
[           'DESISTÊNCIA',             'FINALIZADO',              'CANCELADO',
              'RENOVAÇÃO',                  'ATIVO',                   'NOVO',
 'DEVOLVIDO PARA UNIDADE', 'AGENDADO PARA ATIVAÇÃO',  'AGUARDANDO ASSINATURA',
            'SUBSTITUÍDO',      'AGUARDANDO ATIVAR',           'NAO RENOVADO',
                 'NEGADO',                     <NA>]
Length: 14, dtype: string

In [8]:
# Identificar as placas que possuem pelo menos os status "CANCELADO" e "ATIVO"
placas_com_ambos_status = df_sets.groupby('placa')['status_conjunto'] \
                            .apply(lambda x: {'CANCELADO', 'ATIVO'}.issubset(set(x))) \
                            .loc[lambda x: x].index

# Filtrar o DataFrame completo para manter apenas essas placas
df_sets_filtrado = df_sets[df_sets['placa'].isin(placas_com_ambos_status)]



In [9]:
df_sets_sorted = df_sets.sort_values(['placa', 'data_registro'])

# Função para verificar a sequência ATIVO → CANCELADO → ATIVO
def ativo_cancelado_ativo(grupo):
    status = grupo['status_conjunto'].dropna().astype(str).values
    status_str = ' '.join(status)
    return 'ATIVO CANCELADO ATIVO' in status_str

# Identificar placas que atendem à condição
placas_ativo_cancelado_ativo = df_sets_sorted.groupby('placa').filter(ativo_cancelado_ativo)['placa'].unique()

print("Placas com sequência ATIVO → CANCELADO → ATIVO:")
print(placas_ativo_cancelado_ativo)

Placas com sequência ATIVO → CANCELADO → ATIVO:
<StringArray>
[  'AUA8I08',   'AYC6B19',   'AYZ5A44',   'CSK3I11',   'EZU2H14',   'IVY4D91',
   'NKO1H24',   'NRM7970',   'NRM7971',   'OOM5C63',   'OOM5C64',   'OOM5E81',
   'OOM5E82',   'QAJ3D40',   'QAJ3E40',   'QAU2A09',   'QAU2A19',   'RYJ2E31',
 'SEM-PLACA',   'SWF1H56']
Length: 20, dtype: string


In [10]:
df_sets1 = df_sets[df_sets['placa']=='SWF1H56']
df_sets1.sort_values('data_registro', ascending=False)

,placa,chassi,id_placa,matricula,conjunto,data_registro,data_cancelamento_conjunto,data_ativacao_conjunto,status_conjunto,empresa,data_atualizacao
46340,SWF1H56,9BVRT60D3RE602433,186738,4568,6576,2024-12-05,None,2024-12-05,ATIVO,Viavante,2025-03-18
32128,SWF1H56,9BVRT60D3RE602433,186738,853,4301,2024-10-18,2024-12-05,2024-09-16,CANCELADO,Viavante,2024-12-05
49713,SWF1H56,9BVRT60D3RE602433,186738,853,4204,2024-10-16,None,2024-06-13,ATIVO,Viavante,2025-01-17
40938,SWF1H56,9BVRT60D3RE602433,186738,853,988,2024-06-13,2024-12-05,2024-06-13,CANCELADO,Viavante,2024-12-05
